In [402]:
import random

import numpy as np

### Setup Functions

In [404]:
PATH_TO_INPUT_FILE = '../Sample/sample-input.txt'

def parse_input_file(path):
    puzzles_temp = []
    with open(path, 'r') as f:
        lines_nonempty = filter(None, (line.rstrip() for line in f))
        for line in [line for line in lines_nonempty if line[0] != '#']:
            puzzle_configuration_line = line.split(' ')
            puzzle_state = PuzzleState.construct_initial_puzzle_state(puzzle_configuration_line)
            puzzles_temp.append(puzzle_state)
    return puzzles_temp

puzzles = parse_input_file(PATH_TO_INPUT_FILE)

for puzzle in puzzles:
    print(puzzle.puzzle)
    print(puzzle.car_fuels)


TypeError: 'list' object is not callable

In [ ]:
class PuzzleState:

    def __init__(self, puzzle, car_fuels):
        self.puzzle = puzzle
        self.car_fuels = car_fuels

    @classmethod
    def construct_initial_puzzle_state(cls, puzzle_configuration_line):
        return cls(cls.parse_initial_puzzle(puzzle_configuration_line[0]),
                   cls.parse_initial_car_fuels(puzzle_configuration_line))

    @staticmethod
    def parse_initial_puzzle(puzzle_configuration):
        array_line_input = np.array(list(puzzle_configuration))
        initial_puzzle = np.reshape(array_line_input, (6, 6))
        return initial_puzzle

    @staticmethod
    def parse_initial_car_fuels(puzzle_configuration_line):
        cars = set(puzzle_configuration_line[0].replace('.', ''))
        initial_car_fuels = {}
        for predefined_car_fuel in puzzle_configuration_line[1:]:
            initial_car_fuels[predefined_car_fuel[0]] = int(predefined_car_fuel[1])
        for car in cars.difference(set(initial_car_fuels)):
            initial_car_fuels[car] = 100
        return initial_car_fuels

    # TODO implement this
    def deparse_puzzle_configuration(self):
        return None


class Node:

    def __init__(self, puzzle_state, parent, children):
        self.puzzle_state = puzzle_state
        self.parent = parent
        self.children = children
        self.g = 0
        self.h = 0
        self.f = 0

    def increment_g(self):
        self.g = + 1

    def set_h(self, h):
        self.h = h

    def update_f(self):
        self.f = self.g + self.h





In [ ]:
puzzle = puzzles[0].puzzle
print(puzzle)
print("\n")
# index a row
print(puzzle[0])
# index a column but show as horizontal reg array
print(puzzle[:, 5])
# index a column but show as a vertical column array of 1element arrays
print(puzzle[:, np.array([5])])
# location for ambulance to win
print(puzzle[2, np.array([4,5])])

print("\n")

In [ ]:
# Makes empty new nodes list
# • Gets car length
# • Gets car name
# • Chooses axis
# Chooses direction (ex: left)
# Loops for every period to the left
# Displacement calc
# • Fuel check
# fuel change
# Shifts the car
# Runs valet?
# Saves the node (state, fuels, direction, car name) to his new node list
#
# Repeats for right movement


# it = np.nditer(a, flags=['multi_index'])
# for x in it:
#     print(it.multi_index)

In [ ]:
temp = puzzles[0]
print(temp.puzzle)
print('all cars for ref:', temp.car_fuels)
for car in [car for car in temp.car_fuels if temp.car_fuels[car] > 0]: # only iterate over cars with fuel
    print('car name:', car)
    car_indices = list(zip(*np.where(temp.puzzle == car))) # np.where() returns tuple (rows, cols) so zip to get coords
    print('car indicies:', car_indices)
    length_orientation_vector = np.sum(np.diff(car_indices, axis=0), axis=0) # 2-in-1 to get diff b/w pairs for orientation and sum for length-1
    print(length_orientation_vector)

    current_axis_identifier = -1

    if length_orientation_vector[0] == 0 and length_orientation_vector[1] > 0:
        print('took horizontal path')
        current_axis_identifier = 0
        car_axis = temp.puzzle[car_indices[0][current_axis_identifier], :] # NEEDS TO AXIS DEPENDENT
    elif length_orientation_vector[0] > 0 and length_orientation_vector[1] == 0:
        print('took vertical path')
        current_axis_identifier = 1
        car_axis = temp.puzzle[:, car_indices[0][current_axis_identifier]] # NEEDS TO AXIS DEPENDENT

    print('car axis:', car_axis)
    empty_spot_indices = np.where(car_axis=='.')[0]
    # print('TEST empty_spot_indices without indexing 0:', np.where(car_axis=='.'))
    # print('TEST empty_spot_indices taking [0]:', empty_spot_indices)
    car_indices_one_axis = list(zip(*car_indices))[current_axis_identifier^1]
    # print('TEST car_indices_one_axis but we normally take [0]', list(zip(*car_indices)))
    temp_possible_movements = np.sort(np.concatenate((np.array(car_indices_one_axis), empty_spot_indices)))
    print('periods and the car', temp_possible_movements)
    print('car', car_indices_one_axis)
    car_index_in_temp = np.searchsorted(temp_possible_movements, car_indices_one_axis)
    print('car index in "periods and the car"', car_index_in_temp)
    temp_possible_movements_diff = np.ediff1d(temp_possible_movements)
    print('diff bw poss movements', temp_possible_movements_diff)

    index_temp = min(car_index_in_temp)-1
    while index_temp >= 0:
        if temp_possible_movements_diff[index_temp] == 1:
            # max car index can be at temp_possible_movements[index_temp+1]
            print('car min can be moved to this index in the line:', temp_possible_movements[index_temp])
            # CALCULATE DIFF FOR MOVEMENT DIR AND DISPLACEMENT
            index_temp -= 1
        else:
            break

    index_temp = max(car_index_in_temp)
    while index_temp < np.size(temp_possible_movements_diff):
        if temp_possible_movements_diff[index_temp] == 1:
            # max car index can be at temp_possible_movements[index_temp+1]
            print('car max can be moved to this index in the line:', temp_possible_movements[index_temp+1])
            # CALCULATE DIFF FOR MOVEMENT DIR AND DISPLACEMENT
            index_temp += 1
        else:
            break


    print('\n\n\n\n')

In [ ]:
# [['B' 'B' 'I' 'J' '.' '.']
#  ['.' '.' 'I' 'J' 'C' 'C']
#  ['.' '.' 'I' 'A' 'A' 'M']
#  ['G' 'D' 'D' 'K' '.' 'M']
#  ['G' 'H' '.' 'K' 'L' '.']
#  ['G' 'H' 'F' 'F' 'L' '.']]

car index 1,2
in diff[min(carindex)-1] == 1
in diff[max(carindex)+0] == 1
0 = 1-0
1 = 2-1
2 = 3-2
3 = 4-3
4 = 5-4

max (0, min(carindex)) should let us make sure we dont index into negatives

for the numbers greater than the max index of the car, the max index can be at those numbers
for the numbers less than the min index of the car, the min index of the car can be at those numbers

### Heuristic Functions

In [ ]:
def h1(puzzle):
    index = np.ndarray.flatten(np.argwhere(puzzle.puzzle[2] == 'A'))
    test = puzzle.puzzle[2, index[-1]+1:]
    test = test[test != '.']
    return len(set(test))

# Other solution
# for arr in puzzles:
#     print(arr.puzzle[2])
#     _, idx = np.unique(arr.puzzle[2], return_index=True)
#     temp = arr.puzzle[2][np.sort(idx)]
#     arr_no_duplicates = temp[temp != '.']
#     print((arr_no_duplicates.size - 1) - (np.where(arr_no_duplicates == 'A')[0][0]))

In [ ]:
def h2(puzzle):
    index = np.ndarray.flatten(np.argwhere(puzzle.puzzle[2] == 'A'))
    test = puzzle.puzzle[2, index[-1]+1:]
    test = test[test != '.']
    return len(test)

In [ ]:
LAMBDA = 4

def h3(puzzle):
    return h1(puzzle) * LAMBDA

In [ ]:
def is_goal(puzzle):
    return np.all(puzzle[2, np.array([4,5])] == 'A')

In [ ]:
def is_equal_state(puzzle1, puzzle2):
    return np.array_equal(puzzle1, puzzle2)

In [ ]:
def temp_board(puzzle):
    for element in puzzle:
        print(*element)

In [ ]:
def line_temp_board(puzzle):
    for element in puzzle:
        print(*element, end='', sep='')

In [ ]:
def generate_puzzle():
    f = open("input.txt", "w")

    for x in range(50):
        HORIZONTAL_IDENTIFIER = 0
        temp_board = '....................................'

        #Inserting Ambulance
        a_index = np.random.randint(0,5)
        temp_board = temp_board[: 12 + a_index] + 'AA' + temp_board[a_index + 14 :]

        #Inserting Cars
        car_names = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']
        number_of_cars = np.random.randint(6,13)
        failures = 0
        cars_inserted = 1

        while cars_inserted <= number_of_cars:
            if failures > 10:
                break
            car_length = np.random.randint(2,4)
            car_index = np.random.randint(0, 7 - car_length)
            orientation = np.random.randint(0,2)
            car_line_index = np.random.randint(0,6)
            car_name = car_names[cars_inserted - 1]

            if orientation == HORIZONTAL_IDENTIFIER:
                if not temp_board[6 * car_line_index + car_index : 6 * car_line_index + car_index + car_length].isupper():
                    temp_board = temp_board[: 6 * car_line_index + car_index] + (car_name * car_length) + temp_board[6 * car_line_index + car_index + car_length :]
                    cars_inserted += 1
                else:
                    failures += 1
            else:
                if not (temp_board[car_line_index + 6 * car_index] + temp_board[car_line_index + (6 * car_index) + 6] + (temp_board[car_line_index + (6 * car_index) + 12] if car_length == 3  else '')).isupper():
                    temp_board = temp_board[: car_line_index + (6 * car_index)] + car_name + temp_board[car_line_index + 6 * car_index + 1 : car_line_index + 6 * car_index + 6] + car_name + temp_board[car_line_index + 6 * car_index + 7 : car_line_index + 6 * car_index + 12] + (car_name if car_length == 3  else '') + temp_board[ (car_line_index + 6 * car_index + 13) if car_length == 3 else (car_line_index + 6 * car_index + 12) :]
                    cars_inserted += 1
                else:
                    failures += 1

        #Giving cars fuel
        has_random_amount = np.random.randint(0, cars_inserted/2)
        for y in range(has_random_amount):
            temp_board += " " + str(car_names[np.random.randint(0, cars_inserted)]) + str(np.random.randint(0, 101))

        print(temp_board)
        f.write(temp_board + "\n")

    f.close()

In [ ]:
generate_puzzle()